# Obtain

In [94]:
# load necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


# chosen retail data from https://www.kaggle.com/manjeetsingh/retaildataset


# ori walmart - macos (train.csv = sales data-set.csv)
df_feature = pd.read_csv("/Users/dwihadyan/Downloads/theleadio/ds-exploration/walmart-recruiting-store-sales-forecasting/features.csv", parse_dates=["Date"])
df_sales = pd.read_csv("/Users/dwihadyan/Downloads/theleadio/ds-exploration/walmart-recruiting-store-sales-forecasting/train.csv", parse_dates=["Date"])
df_stores = pd.read_csv("/Users/dwihadyan/Downloads/theleadio/ds-exploration/walmart-recruiting-store-sales-forecasting/stores.csv")
# df_test = pd.read_csv("/Users/dwihadyan/Downloads/theleadio/ds-exploration/walmart-recruiting-store-sales-forecasting/test.csv")




# Scrub

In [96]:
data_date = df_feature.groupby("Date").agg({"Temperature":"mean"
                                            ,"Fuel_Price":"mean"
                                            ,"IsHoliday":"sum"
                                            ,"CPI":"mean"
                                           ,"Unemployment":"mean"})

data_date

,Temperature,Fuel_Price,IsHoliday,CPI,Unemployment
Date,,,,,
2010-02-05,34.037333,2.717844,0,167.730885,8.619311
2010-02-12,34.151333,2.694022,45,167.825608,8.619311
2010-02-19,37.719778,2.672067,0,167.871686,8.619311
2010-02-26,39.243556,2.683933,0,167.909657,8.619311
2010-03-05,42.917333,2.731200,0,167.947628,8.619311
...,...,...,...,...,...
2013-06-28,79.871111,3.629400,0,NaN,NaN
2013-07-05,79.655778,3.583533,0,NaN,NaN
2013-07-12,80.798667,3.568111,0,NaN,NaN


In [97]:
# prepping df_feature & df_sales to be merged, by date
data_date = df_feature.groupby("Date").agg({"Temperature":"mean"
                                            ,"Fuel_Price":"mean"
                                            ,"IsHoliday":"sum"
                                            ,"CPI":"mean"
                                           ,"Unemployment":"mean"})
data_date = data_date.sort_index()
temp_date_data = data_date[:'2012-12-10']
data_sales_date = df_sales.groupby("Date").agg({"Weekly_Sales":"sum"})
data_sales_date.sort_index(inplace=True)

# convert weekly_sales to millions
data_sales_date.Weekly_Sales = data_sales_date.Weekly_Sales/1000000
data_sales_date.Weekly_Sales = data_sales_date.Weekly_Sales.apply(int)

# merge feature & sales tables by date
data = pd.merge(data_sales_date, temp_date_data, left_index=True,right_index=True, how='left')
data["IsHoliday"] = data["IsHoliday"].apply(lambda x: True if x == 45.0 else False )    # 45 because we sum IsHoliday in first line of code in this cell

data

,Weekly_Sales,Temperature,Fuel_Price,IsHoliday,CPI,Unemployment
Date,,,,,,
2010-02-05,49,34.037333,2.717844,False,167.730885,8.619311
2010-02-12,48,34.151333,2.694022,True,167.825608,8.619311
2010-02-19,48,37.719778,2.672067,False,167.871686,8.619311
2010-02-26,43,39.243556,2.683933,False,167.909657,8.619311
2010-03-05,46,42.917333,2.731200,False,167.947628,8.619311
...,...,...,...,...,...,...
2012-09-28,43,68.754444,3.854578,False,176.373588,7.237333
2012-10-05,47,65.973111,3.845222,False,176.505052,6.953711
2012-10-12,46,58.342667,3.896733,False,176.636515,6.953711


In [70]:
# combine df_stores with df_feature w.r.t 'Store' column
data_Store = df_feature.groupby("Store").agg({"Temperature":"mean","Fuel_Price":"mean","IsHoliday":"sum"})

# combine df_stores with df_sales w.r.t 'Store' column
temp_store = df_sales.groupby("Store").agg({"Weekly_Sales":"sum"})

# convert weekly_sales to millions
temp_store.Weekly_Sales = temp_store.Weekly_Sales/1000000
temp_store.Weekly_Sales = temp_store.Weekly_Sales.apply(int)

# set 'Store' as index, and sort data by 'Store' from 1 to 45
data_Store.set_index(np.arange(0,45),inplace=True)

# populate df_stores with data from df_feature & df_sales , w.r.t 'Store' column
df_stores["temp"] = data_Store.Temperature
df_stores["Fuel_Price"] = data_Store.Fuel_Price
df_stores["holiday"] = data_Store.IsHoliday
df_stores["Weekly_Sales"] = temp_store.Weekly_Sales

df_stores = df_stores.dropna()

# Explore


In [71]:
# group sales by month 
data_sales_month = data.groupby(data.index.month).agg({"Weekly_Sales":"sum"})

# group sales by year
data_sales_year = data.groupby(data.index.year).agg({"Weekly_Sales":"sum"})

In [72]:
# group sales by dept
data_Dept = df_sales.groupby("Dept").agg({"Weekly_Sales":"sum"})

# convert sales to millions & sort by sales lowest to highest
data_Dept.Weekly_Sales = data_Dept.Weekly_Sales/10000
data_Dept.Weekly_Sales = data_Dept.Weekly_Sales.apply(int)
data_Dept.sort_values(by="Weekly_Sales")

,Weekly_Sales
Dept,
39,0
78,0
43,0
47,0
51,3
...,...
90,29106
72,30572
38,39311


In [73]:
# group sales by date & store
sales_date_store = df_sales.groupby(["Date","Store"]).agg({"Weekly_Sales":"sum"})

# sort by index
sales_date_store.sort_index(inplace=True)

# convert sales to millions
sales_date_store.Weekly_Sales = sales_date_store.Weekly_Sales/10000
sales_date_store.Weekly_Sales = sales_date_store.Weekly_Sales.apply(int)

# merge df_feature & df that group data by date_year & store_type 
data_table = pd.merge(df_feature,sales_date_store ,  how='left', on=["Date","Store"])
data_table = pd.merge(data_table,df_stores[["Store","Type"]] ,  how='left', on=["Store"])
data_table.head(20)
data_train = data_table[data_table.Weekly_Sales.notnull()]
data_test = data_table[data_table.Weekly_Sales.isnull()]

data_table

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Weekly_Sales,Type
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,164.0,NaN
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,164.0,NaN
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,161.0,NaN
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,140.0,NaN
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,155.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False,NaN,B
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False,NaN,B
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False,NaN,B
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False,NaN,B


In [74]:
# # Analysis of each markdown
# markdown : discount given to the product (in value).
# markdown1-5 is different types of discount. (can be anything, specifics never mentioned)
# if its negative, means its markup price
# eg: m1 general discount, m2 membership discount, m3 holiday discount, m4 elderly discount, m5 children discount
# if price $160, given 50% general discount, hence markdown1 is $80. 
# given membership disc 50% on top, hence markdown2 is $40
# no more discount, hence markdown3, markdown4, markdown5 is 0


# group markdown by daily mean
train_markdown = data_table[data_table.MarkDown2.notnull()]
train_markdown = train_markdown.groupby("Date").agg({"MarkDown1":"mean","MarkDown2":"mean","MarkDown3":"mean","MarkDown4":"mean","MarkDown5":"mean"})




In [75]:
# group markdown by month mean
train_markdown_month = train_markdown.groupby(train_markdown.index.month).agg({"MarkDown1":"mean","MarkDown2":"mean","MarkDown3":"mean","MarkDown4":"mean","MarkDown5":"mean"})

In [76]:
# group markdown by typeshop
train_markdown_1 = data_table[data_table.MarkDown2.notnull()]
train_markdown_type = train_markdown_1.groupby("Type").agg({"MarkDown1":"mean","MarkDown2":"mean","MarkDown3":"mean","MarkDown4":"mean","MarkDown5":"mean"})


# Model

- Using multiple models to see which has the best accuracy (loop through different models)

In [77]:
from fancyimpute import IterativeImputer
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.linear_model import ElasticNet, Lasso, RidgeCV,LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor

In [78]:
# create dummies & plug it in the main table

def createdummies(data,cols):
    for col in cols:
        one_hot = pd.get_dummies(data[col],prefix=col)
        data = data.join(one_hot)
        data.drop(col,axis = 1,inplace=True)
    
    return data

In [79]:
# imputing the missing value
itt = IterativeImputer()
df = itt.fit_transform(data_table[["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5"]]) 
data_table.MarkDown1 = df[:,0]
data_table.MarkDown2 = df[:,1]
data_table.MarkDown3 = df[:,2]
data_table.MarkDown4 = df[:,3]
data_table.MarkDown5 = df[:,4]

data_table['CPI'].fillna((data_table['CPI'].mean()), inplace=True)
data_table['Unemployment'].fillna((data_table['Unemployment'].mean()), inplace=True)
data_table['IsHoliday'] = data_table['IsHoliday'].map({True:0,False:1})

#create new column
data_table["Month"] = data_table.Date.dt.month
data_table["Year"] = data_table.Date.dt.year
data_table["WeekofYear"] = data_table.Date.dt.weekofyear
data_table.drop(['Date'],axis=1,inplace=True)

#create dummies out of categorical column
data_table = createdummies(data_table,["Type","Month","Year","WeekofYear"])

data_table

/var/folders/29/_r682w3n2m95fqd9bv6_hk300000gn/T/ipykernel_2405/4128095892.py:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data_table["WeekofYear"] = data_table.Date.dt.weekofyear


,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,...,WeekofYear_43,WeekofYear_44,WeekofYear_45,WeekofYear_46,WeekofYear_47,WeekofYear_48,WeekofYear_49,WeekofYear_50,WeekofYear_51,WeekofYear_52
0,1,42.31,2.572,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.096358,8.106000,...,0,0,0,0,0,0,0,0,0,0
1,1,38.51,2.548,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.242170,8.106000,...,0,0,0,0,0,0,0,0,0,0
2,1,39.93,2.514,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.289143,8.106000,...,0,0,0,0,0,0,0,0,0,0
3,1,46.63,2.561,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.319643,8.106000,...,0,0,0,0,0,0,0,0,0,0
4,1,46.50,2.625,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.350143,8.106000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,76.05,3.639,4842.290000,975.030000,3.000000,2449.970000,3169.690000,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8186,45,77.50,3.614,9090.480000,2268.580000,582.740000,5797.470000,1514.930000,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8187,45,79.37,3.614,3789.940000,1827.310000,85.720000,744.840000,2150.360000,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8188,45,82.84,3.737,2961.490000,1047.070000,204.190000,363.000000,1059.460000,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# data split 

data_train = data_table[data_table.Weekly_Sales.notnull()]
data_test = data_table[data_table.Weekly_Sales.isnull()]
X = data_train.drop('Weekly_Sales', axis=1)
y = data_train['Weekly_Sales']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [81]:
# run multiple models. WARNING : takes 1min+ to run this cell

classifiers = [
    LinearRegression(),
    # ElasticNet(),
    # RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]),
    # KernelRidge(alpha=0.6, kernel='polynomial', degree=3, coef0=2.5),
    # Lasso(alpha =16, random_state=100),
    # ElasticNet(alpha=0.8),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    AdaBoostRegressor()
    # SVR(), 
    # LinearSVR(), 
    # NuSVR()
    ]

name = []
score = []
models = []
rmse = []

i = 0
for classifier in classifiers:

    # train model
    classifier.fit(X_train, y_train)

    # test model   
    name.append(type(classifier).__name__)
    score.append(classifier.score(X_test, y_test))
    models.append(classifier)
    rmse.append(np.sqrt(mean_squared_error(classifier.predict(X_test), y_test)))

In [82]:
# plot result of each model performance

# rmse : root mean square errors (lower better)
# score : accuracy score (higher better)

df_score = pd.DataFrame(list(zip(name,rmse, score, models)),columns=['name','rmse','score',"model"])
df_score.set_index('name',inplace=True)
df_score.sort_values(by=['score'],inplace=True)
df_score

,rmse,score,model
name,,,
LinearRegression,38.872015,0.530004,LinearRegression()
AdaBoostRegressor,35.571061,0.606437,"(DecisionTreeRegressor(max_depth=3, random_sta..."
GradientBoostingRegressor,14.636406,0.933367,([DecisionTreeRegressor(criterion='friedman_ms...
DecisionTreeRegressor,13.133497,0.946349,DecisionTreeRegressor()
RandomForestRegressor,11.589888,0.958219,"(DecisionTreeRegressor(max_features='auto', ra..."


# iNterpret

In [83]:
# prediction using best model (randomforest)

model = df_score.loc["RandomForestRegressor","model"]
data_test.drop(['Weekly_Sales'],axis=1,inplace=True)
predict = model.predict(data_test)
predict

/Users/dwihadyan/opt/anaconda3/envs/quantra38/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


array([163.25, 162.11, 158.91, ...,  79.45,  80.41,  87.59])

In [84]:
submission = pd.read_csv("/Users/dwihadyan/Downloads/theleadio/ds-exploration/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv")
df_test = pd.read_csv("/Users/dwihadyan/Downloads/theleadio/ds-exploration/walmart-recruiting-store-sales-forecasting/test.csv", parse_dates=["Date"])

df_test.sample(3)

,Store,Dept,Date,IsHoliday
76172,29,18,2013-07-05,False
13415,6,3,2013-06-28,False
85413,32,96,2013-04-19,False


In [85]:
df_test["Month"] = df_test.Date.dt.month
df_test["Year"] = df_test.Date.dt.year
df_test["WeekofYear"] = df_test.Date.dt.weekofyear
df_test.drop(['Date'],axis=1,inplace=True)


#create dummies out of categorical column
df_test = createdummies(df_test,["Month","Year","WeekofYear"])
df_test

/var/folders/29/_r682w3n2m95fqd9bv6_hk300000gn/T/ipykernel_2405/96388345.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_test["WeekofYear"] = df_test.Date.dt.weekofyear


,Store,Dept,IsHoliday,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,...,WeekofYear_30,WeekofYear_44,WeekofYear_45,WeekofYear_46,WeekofYear_47,WeekofYear_48,WeekofYear_49,WeekofYear_50,WeekofYear_51,WeekofYear_52
0,1,1,False,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,False,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,1,False,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,1,True,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1,1,False,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,False,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
115060,45,98,False,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
115061,45,98,False,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
115062,45,98,False,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [88]:
data_table

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,...,WeekofYear_43,WeekofYear_44,WeekofYear_45,WeekofYear_46,WeekofYear_47,WeekofYear_48,WeekofYear_49,WeekofYear_50,WeekofYear_51,WeekofYear_52
0,1,42.31,2.572,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.096358,8.106000,...,0,0,0,0,0,0,0,0,0,0
1,1,38.51,2.548,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.242170,8.106000,...,0,0,0,0,0,0,0,0,0,0
2,1,39.93,2.514,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.289143,8.106000,...,0,0,0,0,0,0,0,0,0,0
3,1,46.63,2.561,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.319643,8.106000,...,0,0,0,0,0,0,0,0,0,0
4,1,46.50,2.625,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.350143,8.106000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,76.05,3.639,4842.290000,975.030000,3.000000,2449.970000,3169.690000,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8186,45,77.50,3.614,9090.480000,2268.580000,582.740000,5797.470000,1514.930000,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8187,45,79.37,3.614,3789.940000,1827.310000,85.720000,744.840000,2150.360000,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8188,45,82.84,3.737,2961.490000,1047.070000,204.190000,363.000000,1059.460000,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0


In [89]:
data_train

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,...,WeekofYear_43,WeekofYear_44,WeekofYear_45,WeekofYear_46,WeekofYear_47,WeekofYear_48,WeekofYear_49,WeekofYear_50,WeekofYear_51,WeekofYear_52
0,1,42.31,2.572,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.096358,8.106,...,0,0,0,0,0,0,0,0,0,0
1,1,38.51,2.548,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.242170,8.106,...,0,0,0,0,0,0,0,0,0,0
2,1,39.93,2.514,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.289143,8.106,...,0,0,0,0,0,0,0,0,0,0
3,1,46.63,2.561,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.319643,8.106,...,0,0,0,0,0,0,0,0,0,0
4,1,46.50,2.625,7031.852089,3391.983532,1835.828065,2675.211153,4132.216422,211.350143,8.106,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8146,45,64.88,3.997,4556.610000,20.640000,1.500000,1601.010000,3288.250000,192.013558,8.684,...,0,0,0,0,0,0,0,0,0,0
8147,45,64.89,3.985,5046.740000,3342.703674,18.820000,2253.430000,2340.010000,192.170412,8.667,...,0,0,0,0,0,0,0,0,0,0
8148,45,54.47,4.000,1956.280000,3271.486251,7.890000,599.320000,3990.540000,192.327265,8.667,...,0,0,0,0,0,0,0,0,0,0
8149,45,56.47,3.969,2004.020000,3315.320248,3.180000,437.730000,1537.490000,192.330854,8.667,...,0,0,0,0,0,0,0,0,0,0


In [90]:
data_test

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,...,WeekofYear_43,WeekofYear_44,WeekofYear_45,WeekofYear_46,WeekofYear_47,WeekofYear_48,WeekofYear_49,WeekofYear_50,WeekofYear_51,WeekofYear_52
143,1,55.32,3.386,6766.44,5147.700000,50.82,3639.90,2737.42,223.462779,6.573000,...,0,1,0,0,0,0,0,0,0,0
144,1,61.24,3.314,11421.32,3370.890000,40.28,4646.79,6154.16,223.481307,6.573000,...,0,0,1,0,0,0,0,0,0,0
145,1,52.92,3.252,9696.28,292.100000,103.78,1133.15,6612.69,223.512911,6.573000,...,0,0,0,1,0,0,0,0,0,0
146,1,56.23,3.211,883.59,4.170000,74910.32,209.91,303.32,223.561947,6.573000,...,0,0,0,0,1,0,0,0,0,0
147,1,52.34,3.207,2460.03,3215.769411,3838.35,150.57,6966.34,223.610984,6.573000,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,76.05,3.639,4842.29,975.030000,3.00,2449.97,3169.69,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8186,45,77.50,3.614,9090.48,2268.580000,582.74,5797.47,1514.93,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8187,45,79.37,3.614,3789.94,1827.310000,85.72,744.84,2150.36,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0
8188,45,82.84,3.737,2961.49,1047.070000,204.19,363.00,1059.46,172.460809,7.826821,...,0,0,0,0,0,0,0,0,0,0


In [98]:
submission

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0
...,...,...
115059,45_98_2013-06-28,0
115060,45_98_2013-07-05,0
115061,45_98_2013-07-12,0
115062,45_98_2013-07-19,0


In [101]:
submission

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0
...,...,...
115059,45_98_2013-06-28,0
115060,45_98_2013-07-05,0
115061,45_98_2013-07-12,0
115062,45_98_2013-07-19,0
